In [5]:
# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    readr,
#    reshape2,
    SnowballC,
    stopwords,
    stringr,
    textclean,
#    tidytext,
#    tidyverse,
    tm,
    textreg,
#    topicdoc,
#    topicmodels,
    qdap,
    qdapRegex
)

In [6]:
data <- read_csv('./data/generated/debotted_combined_no_rts.csv')

Rows: 80280 Columns: 1
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): content

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
clusters <- unique(data$cluster)

Warning message:
“Unknown or uninitialised column: `cluster`.”


In [9]:
clean_and_tokenize <- function(text, language){
    payload <- text %>%
        str_replace('^RT', '') %>%
        str_to_lower() %>%
        replace_contraction() %>%
        #replace_emoji() # replacements have spaces
        #replace_emoticons()
        str_replace_all(pattern = '#([a-z0-9])', replacement = function(text){paste0('hashtag',gsub('^#', '', text),'hashtag')}) %>% # place 'hashtag' before and after to circumvent punctuation filter and stemming
        rm_url() %>%
        str_remove_all(pattern = '@\\w+') %>%
        str_remove_all(pattern = '\\$?\\d+\\w*') %>%
        str_replace_all(pattern = '[[:punct:]]', replacement = '') %>%
        str_squish() %>%
        str_trim() %>%
        stemDocument(language = language) %>%
        qdap::rm_stopwords(tm::stopwords(language)) %>%
        sapply(paste, collapse=" ") %>%
        str_replace_all(pattern = 'hashtag(\\w+)hashtag', replacement = '#\\1') # convert hashtags back


    # convert corpus to document term matrix
    return(payload);
}

for(cluster_name in clusters){
    cluster_data <- data %>% filter(cluster == cluster_name)
    tokenized <- clean_and_tokenize(cluster_data$content, 'english')
    cluster_data.tokenized <- cluster_data %>% mutate(tokenized = tokenized)
    cluster_data.tokenized %>% write_csv(paste0('./data/generated/data_with_tokenized_no_rts_', cluster_name, '.csv'))
    cat(tokenized %>% paste(collapse = '\n'), file = paste0('./data/generated/tokenized_no_rts_', cluster_name, '.txt'))
    ### write file
}